In [31]:
!pip install setfit==1.1.0
!pip install transformers==4.42.2
!mkdir models


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\durek\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\durek\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
A subdirectory or file models already exists.


In [32]:
import pandas as pd
import time
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset, DatasetDict, load_dataset
from tqdm.auto import tqdm
import numpy as np
import torch

tqdm.pandas()

In [33]:
# dataset from hf_hub
langs = ['java', 'python', 'pharo']
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = load_dataset('NLBSE/nlbse25-code-comment-classification')
ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

In [34]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))
stop_words.add(',')

def remove_stop_words(data):
    sentences = nltk.word_tokenize(data['combo'])
    filtered_sentence = [word for word in sentences if word not in stop_words]
    data['combo'] = ' '.join(filtered_sentence)
    
    return data

for lang in langs:
    ds[f'{lang}_train'] = ds[f'{lang}_train'].map(remove_stop_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\durek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\durek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [35]:
for lan in langs:
    model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L3-v2", multi_target_strategy="multi-output", device="cuda")
    trainer = SetFitTrainer(
        model=model,
        train_dataset=ds[f'{lan}_train'],
        column_mapping={"combo": "text", "labels": "label"},
        num_epochs=2 if lan == 'java' else 5,
        batch_size=32,
    )
    trainer.train()
    trainer.model.save_pretrained(f'./models/{lan}')

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
C:\Users\durek\AppData\Local\Temp\ipykernel_27004\3106613784.py:3: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset






Map: 100%|██████████| 7614/7614 [00:00<00:00, 34598.81 examples/s]
***** Running training *****
  Num unique pairs = 304560
  Batch size = 32
  Num epochs = 2




                                                      
                                                  

                                             


  2%|▏         | 426/19036 [1:01:20<44:53:26,  8.68s/it]









{'embedding_loss': 0.2564, 'grad_norm': 1.526802897453308, 'learning_rate': 1.0504201680672269e-08, 'epoch': 0.0}


  0%|          | 5/19036 [00:00<33:03,  9.59it/s]





























                                                        
                                                  

                                             


  2%|▏         | 426/19036 [1:01:22<44:53:26,  8.68s/it]






{'embedding_loss': 0.2891, 'grad_norm': 1.466935634613037, 'learning_rate': 5.252100840336135e-07, 'epoch': 0.01}
































                                                        
                                                  

                                             


  2%|▏         | 426/19036 [1:01:24<44:53:26,  8.68s/it]








...

{'embedding_loss': 0.0143, 'grad_norm': 0.6182732582092285, 'learning_rate': 1.7800903738189785e-07, 'epoch': 4.96}






























                                                        
                                                  

                                             


  2%|▏         | 426/19036 [1:26:42<44:53:26,  8.68s/it]








{'embedding_loss': 0.0169, 'grad_norm': 0.6660462021827698, 'learning_rate': 4.107900862659182e-08, 'epoch': 4.99}








                                                        
                                                  

                                             


  2%|▏         | 426/19036 [1:26:43<44:53:26,  8.68s/it]


100%|██████████| 8115/8115 [04:45<00:00, 28.39it/s]


{'train_runtime': 285.7866, 'train_samples_per_second': 908.37, 'train_steps_per_second': 28.395, 'train_loss': 0.04567978508811781, 'epoch': 5.0}


In [36]:
total_flops = 0
total_time = 0
scores = []
for lan in langs:
    # to load trained models:
    model = SetFitModel.from_pretrained(f'./models/{lan}')
    # to load pretrained models from Hub:
    # model = SetFitModel.from_pretrained(f"NLBSE/nlbse25_{lan}")
    with torch.profiler.profile(with_flops=True) as p:
        begin = time.time()
        for i in range(10):
          y_pred = model(ds[f'{lan}_test']['combo']).numpy().T
        total = time.time() - begin
        total_time = total_time + total
    total_flops = total_flops + (sum(k.flops for k in p.key_averages()) / 1e9)
    y_true = np.array(ds[f'{lan}_test']['labels']).T
    for i in range(len(y_pred)):
        assert(len(y_pred[i]) == len(y_true[i]))
        tp = sum([true == pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
        tn = sum([true == pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
        fp = sum([true == 0 and pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
        fn = sum([true == 1 and pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = (2*tp) / (2*tp + fp + fn)
        scores.append({'lan': lan, 'cat': labels[lan][i],'precision': precision,'recall': recall,'f1': f1})
print("Compute in GFLOPs:", total_flops/10)
print("Avg runtime in seconds:", total_time/10)
scores = pd.DataFrame(scores)
scores

Compute in GFLOPs: 1997.2684032000002
Avg runtime in seconds: 0.596540093421936


,lan,cat,precision,recall,f1
0,java,summary,0.896163,0.890135,0.893138
1,java,Ownership,1.000000,1.000000,1.000000
2,java,Expand,0.304348,0.480392,0.372624
3,java,usage,0.933687,0.816705,0.871287
4,java,Pointer,0.799107,0.972826,0.877451
5,java,deprecation,0.625000,0.666667,0.645161
6,java,rational,0.333333,0.323529,0.328358
7,python,Usage,0.790000,0.652893,0.714932
8,python,Parameters,0.753521,0.835938,0.792593
9,python,DevelopmentNotes,0.217742,0.658537,0.327273


In [37]:
max_avg_runtime = 5
max_avg_flops = 5000

# s𝑢𝑏𝑚𝑖𝑠𝑠𝑖𝑜𝑛_𝑠𝑐𝑜𝑟𝑒(𝑚𝑜𝑑𝑒𝑙)=(𝑎𝑣𝑔. 𝐹1)×0.60+((𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)/𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)×0.2+((𝑚𝑎𝑥_GFLOPs−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_GFLOPs)/𝑚𝑎𝑥_GFLOPs)×0.2
# Base: 0.73
# Run 1 (Removed Stopwords): 0.72
# Run 2 (Removed stopwords to combo): 0.70
def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
      0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
      0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

avg_f1 = scores.f1.mean()
avg_runtime = total_time/10
avg_flops = total_flops/10

round(score(avg_f1, avg_runtime, avg_flops), 2)

0.65